In [284]:
import pymysql.cursors
from cryptography.fernet import Fernet
import secrets
import string
from getpass import getpass
import sys
import pyperclip
import tkinter as tk
from tkinter import messagebox

# Database Connection

In [94]:
connect_info = {
    'host':'127.0.0.1',
    'port':3306,
    'user':'root',
    'password':'890521Willy',
    'db':'Account'
}

connection = pymysql.connect(**connect_info)
cursor = connection.cursor()

# Golden KEY

In [95]:
with open('GoldenKEY.txt') as GoldenFile:
    KEY = GoldenFile.read()
    
CodeKEY = Fernet(KEY)

# View generate

service_code 操作選擇  
type_code 分類選擇  

In [293]:
class main_control(tk.Tk):
    
    def __init__(self):
        tk.Tk.__init__(self)
        self.title("Account Vault")
        self.geometry("400x600")
        self._frame = None
        self.switch_frame(main_menu)

    def switch_frame(self, frame_class, **kwagrs):
        new_frame = frame_class(self, kwagrs)
        if self._frame is not None:
            self._frame.destroy()
        self._frame = new_frame
        self._frame.pack()
        
        

In [294]:
class login_page(tk.Frame):
    
    def __init__(self, controler, kwargs):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        tk.Label(self, text="Welcome to Account Vault", font=("Times",20)).pack(side="top")
        tk.Label(self, text="Please Enter Password", font=("Times",20)).pack(side="top")
        self.password = tk.Entry(self,show="*")
        self.password.pack()
        tk.Button(self, text="Login", command=self.validate).pack()
    
    def validate(self):
        password = self.password.get()
        #get ultimate password
        demand = "SELECT Password FROM Account_info WHERE Institution = 'Ultimate'"
        cursor.execute(demand)
        result = cursor.fetchone()[0]
        decrypted = CodeKEY.decrypt(result.encode())
        
        if password == decrypted.decode():
            self.controler.switch_frame(main_menu)

In [295]:
class main_menu(tk.Frame):
    
    def __init__(self, controler, kwargs):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        tk.Label(self, text="Welcome Back Master", font=("Times",15)).pack(side="top")
        tk.Label(self, text="What are you looking for today?", font=("Times",20)).pack(side="top")
        tk.Button(self, text="Get Account", command=lambda: self.service_change("1"), height=5, width=20).pack()
        tk.Button(self, text="Add Account", command=lambda: self.service_change("2"), height=5, width=20).pack()
        tk.Button(self, text="Update Account", command=lambda: self.service_change("3"),height=5, width=20).pack()
        #tk.Button(self, text="Exit", command=lambda: self.service_change("4"),height=5, width=20).pack()
    def service_change(self, service_code):
        self.controler.switch_frame(get_type, service_code=service_code)

In [296]:
class get_type(tk.Frame):
    def __init__(self, controler, kwargs):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        tk.Label(self, text="Get Account", font=("Times",20)).pack(side="top")
        tk.Label(self, text="What Type of Account Are You Looking For?", font=("Times",15)).pack(side="top")
        
        tk.Button(self, text="GAME", height=5, width=20, command=lambda: self.type_change("1", kwargs["service_code"])).pack()
        tk.Button(self, text="SHOP", height=5, width=20, command=lambda: self.type_change("2", kwargs["service_code"])).pack()
        tk.Button(self, text="WORK", height=5, width=20, command=lambda: self.type_change("3", kwargs["service_code"])).pack()
        tk.Button(self, text="STUDY", height=5, width=20, command=lambda: self.type_change("4", kwargs["service_code"])).pack()
        
    def type_change(self, type_code, service_code):
        if service_code == "1":
            self.controler.switch_frame(get_account_serive, type_code=type_code ,service_code=service_code)
        
class get_account_serive(tk.Frame):
    def __init__(self, controler, kwargs):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        if kwargs['type_code'] == "1":
            tk.Label(self, text="GAME", font=("Times",20)).grid(row=0,column=0)
        elif kwargs['type_code'] == "2":
            tk.Label(self, text="SHOP", font=("Times",20)).grid(row=0,column=0)
        elif kwargs['type_code'] == "3":
            tk.Label(self, text="WORK", font=("Times",20)).grid(row=0,column=0)
        elif kwargs['type_code'] == "4":
            tk.Label(self, text="STUDY", font=("Times",20)).grid(row=0,column=0)
            
        tk.Label(self, text="Institutions in This Category", font=("Times",15)).grid(row=1,column=0)
        #organize frame for button
        institution_frame = tk.Frame(self)
        institution_frame.grid(row=3,column=0)
        # get insitutions
        demand = f"SELECT Institution FROM Account_info WHERE Category = {kwargs['type_code']}"
        cursor.execute(demand)
        result = cursor.fetchall()
        #organize button 10 in one column
        row=0
        column=0
        for item in result:
            tk.Button(institution_frame, text=item[0], height=2, width=15, 
                      command=lambda item=item: self.show_account(item[0],kwargs['type_code'])).grid(row=row,column=column)
            row+=1
            if row % 10 ==0:
                column+=1
                row = 0
        tk.Button(self, text="Back to Last Page", height=2, width=15, 
                  command=lambda: self.controler.switch_frame(get_type, service_code="1")).grid(row=4, column=0)  
        
    def show_account(self, institution, type_code):
        self.controler.switch_frame(show_all_account, ins=institution, type_code=type_code)

In [305]:
class show_all_account(tk.Frame):
    def __init__(self, controler, kwargs):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        institution = kwargs['ins']
        tk.Label(self, text=f"Account in {institution}", font=("Times",20)).grid(row=0,column=0)
        #organize button frame
        account_frame = tk.Frame(self)
        account_frame.grid(row=1,column=0)
        #get all account
        institution = "'" + institution + "'"
        demand = f'SELECT Account FROM Account_info WHERE Institution = {institution}'
        cursor.execute(demand)
        result = cursor.fetchall()
        #organize button
        row=0
        column=0
        for item in result:
            tk.Button(account_frame, text=item[0], height=2, width=15, 
                      command=lambda account=item[0], institution=institution: self.get_info(account,institution)).grid(row=row,column=column)
            row+=1
            if row % 10 ==0:
                column+=1
                row = 0
        tk.Button(self, text="Back to Last Page", height=2, width=15, 
                  command=lambda: self.controler.switch_frame(get_account_serive, type_code=kwargs["type_code"])).grid(row=4, column=0)
    def get_info(self, account, institution):
        account = "'" + account + "'"
        demand = f'SELECT Account, Password FROM Account_info WHERE Account = {account} AND Institution = {institution}'
        cursor.execute(demand)
        result = cursor.fetchone()
        password = CodeKEY.decrypt(result[1].encode()).decode()
        
        messagebox.showinfo("Account Info", f"Account: {result[0]}\nPassword: {password}")

In [306]:
class add_account(tk.Frame):
    def __init__(self, controler):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        tk.Label(self, text="Add Account", font=("Times",15)).pack(side="top")

In [307]:
class update_account(tk.Frame):
    def __init__(self, controler):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        tk.Label(self, text="Update Account", font=("Times",15)).pack(side="top")

In [308]:
app = main_control()
app.mainloop()